In [1]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg
from mv_wg_avg import exp_weight_avg

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
b_id = 'SK_ID_BUREAU'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV']

2018-08-10 16:39:45,182 logger 19 [INFO]    [logger_func] start 
2018-08-10 16:39:45,206 logger 19 [INFO]    [logger_func] start 
2018-08-10 16:39:45,206 logger 19 [INFO]    [logger_func] start 


In [34]:
base = pd.read_csv('../data/base.csv')[[unique_id, target]]
# app = pd.read_csv('../data/application_train_test_after.csv')
bureau = pd.read_csv('../data/bureau_cleansing.csv')
bureau['DAYS_ENDDATE'] = bureau.apply(lambda x: x['DAYS_ENDDATE_FACT'] if x['DAYS_ENDDATE_FACT'] < 99999 else x['DAYS_CREDIT_ENDDATE'], axis=1)

prev = pd.read_csv('../data/previous_cleansing.csv')

# balance = pd.read_csv('../data/bureau_balance.csv')
# balance['STATUS'] =  balance['STATUS'].map(lambda x: '-1' if x=='C' or x=='X' else x).astype('int')
# b_status = balance.groupby(b_id)['STATUS'].max()
# b_status = b_status.reset_index().query('STATUS>=0')
# bureau = bureau.merge(b_status, on=b_id, how='left')

# pos['DPD_CODE'] = pos['SK_DPD'].map(lambda x: 'DPD' if x>0 else 'NAN')
# pos['DPD_DEF_CODE'] = pos['SK_DPD_DEF'].map(lambda x: 'DPD_DEF' if x>0 else 'NAN')
# ccb['DPD_CODE'] = ccb['SK_DPD'].map(lambda x: 'DPD' if x>0 else 'NAN')
# ccb['DPD_DEF_CODE'] = ccb['SK_DPD_DEF'].map(lambda x: 'DPD_DEF' if x>0 else 'NAN')

SystemExit: 

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [49]:
bureau['AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
amt_list = [col for col in bureau.columns if col.count('AMT_')]
days_list = [col for col in bureau.columns if col.count('DAYS_')]
print(amt_list)
print(days_list)

amt_sum = bureau[[unique_id] + amt_list].groupby(unique_id).sum().reset_index()
days_max = bureau[[unique_id] + days_list].groupby(unique_id).max().reset_index()

for col in amt_sum[amt_list].columns:
    amt_sum.rename(columns={col:f'{col}_sum'}, inplace=True)
    
for col in days_max[days_list].columns:
    days_max.rename(columns={col:f'{col}_max'}, inplace=True)

['AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_ANNUITY', 'AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT']
['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE', 'DAYS_ENDDATE']


In [50]:
prefix = 'b_'
label = 'DAYS_CREDIT'
data = bureau[[unique_id, label, 'DAYS_ENDDATE'] + amt_list].copy()
data['BUREAU_CREDIT_TERM'] = data['DAYS_ENDDATE'] - data[label]

' 重み付き平均 '
label = 'DAYS_ENDDATE'
weight_list = [0.99]
feature_list = amt_list + ['BUREAU_CREDIT_TERM']

df_wmean = bureau[unique_id].drop_duplicates().to_frame().copy()

for num in feature_list:
    for weight in weight_list:
        df = data[[unique_id, label, num]].dropna().copy()
        wavg = exp_weight_avg(data=df, level=unique_id, weight=weight, label=label, value=num)
#         result = base.merge(wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
        df_wmean = df_wmean.merge(wavg.to_frame().reset_index(), on=unique_id, how='left')
#         make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)
print(df_wmean.shape)
display(df_wmean.head(20))
del data

(176053, 9)


,SK_ID_CURR,AMT_CREDIT_MAX_OVERDUE_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_DEBT_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_LIMIT_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_OVERDUE_wg0.99_avg@SK_ID_CURR,AMT_ANNUITY_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT_wg0.99_avg@SK_ID_CURR,BUREAU_CREDIT_TERM_wg0.99_avg@SK_ID_CURR
0,179735,7.920000,52969.797290,0.000000,0.000000,7.920000,9.000000,48614.013741,20.883935
1,114166,34032.240000,286148.022394,0.000000,0.000000,113855.563666,35105.940000,174851.235000,28.134395
2,154094,125.550000,221958.013436,39246.469142,17131.518323,125.550000,3462.435000,177251.898104,19.707374
3,127520,5800.500000,231380.418245,158998.949871,0.000000,7.560000,4999.995000,131697.177778,66.213789
4,299541,9783.270000,84756.558975,25616.729752,0.000000,321.750000,10032.030000,59139.829222,10.198350
5,210690,228.690000,784129.785844,599373.252662,0.000000,228.690000,19438.636734,124888.452827,37.129115
6,208307,4015.841073,101115.830637,0.000000,0.000000,0.270000,3241.620000,108077.498350,14.518100
7,257173,51.840000,63667.139739,0.000000,0.000000,51.840000,3225.240000,63667.139739,6.425998
8,351898,18335.705524,406925.915609,308374.979203,0.000000,53058.219195,24830.544973,195415.516192,33.990869
9,236523,5585.940000,70219.175505,14163.958292,0.000000,5585.940000,6394.500000,53535.951600,11.180193


In [53]:
bureau_imp_base = amt_sum.merge(days_max, on=unique_id, how='left').merge(df_wmean, on=unique_id, how='left')
result_bureau = base.merge(bureau_imp_base, on=unique_id, how='left')
result_bureau.to_csv('../data/impute_before_bureau.csv', index=False)
result_bureau.head()

,SK_ID_CURR,TARGET,AMT_CREDIT_MAX_OVERDUE_sum,AMT_CREDIT_SUM_sum,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_LIMIT_sum,AMT_CREDIT_SUM_OVERDUE_sum,AMT_ANNUITY_sum,AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT_sum,DAYS_CREDIT_max,DAYS_CREDIT_ENDDATE_max,DAYS_ENDDATE_FACT_max,DAYS_CREDIT_UPDATE_max,DAYS_ENDDATE_max,AMT_CREDIT_MAX_OVERDUE_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_DEBT_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_LIMIT_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_OVERDUE_wg0.99_avg@SK_ID_CURR,AMT_ANNUITY_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT_wg0.99_avg@SK_ID_CURR,BUREAU_CREDIT_TERM_wg0.99_avg@SK_ID_CURR
0,100002,1,8405.145,347555.565,0.0,31988.565,0.0,0.0,287723.565,-3.0,2.0,-1.0,0.0,-1.0,4306.961833,81457.692829,0.0,0.0,NaN,NaN,126521.820426,11.294591
1,100003,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0.000,189037.800,0.0,0.000,0.0,0.0,189037.800,-13.0,-12.0,-12.0,-12.0,-12.0,NaN,94519.848958,0.0,0.0,NaN,NaN,94519.848958,10.972801
3,100006,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
prev

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,TARGET,valid_no,is_train,is_test
0,2462805,299809,Cash loans,9842.400,90000.000,90000.00,NaN,90000.000,THURSDAY,13,Y,1.0,NaN,NaN,NaN,XNA,Approved,-75.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash Street: high,-75.0,-74.0,-63.0,-63.0,-63.0,0.0,0,3.0,1,0
1,1626275,426902,Cash loans,5246.010,45000.000,47970.00,NaN,45000.000,THURSDAY,15,Y,1.0,NaN,NaN,NaN,XNA,Approved,-74.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash Street: high,-74.0,-73.0,-62.0,-62.0,-62.0,1.0,0,3.0,1,0
2,1983905,157753,Cash loans,8610.120,112500.000,127350.00,NaN,112500.000,THURSDAY,14,Y,1.0,NaN,NaN,NaN,XNA,Approved,-74.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,24.0,high,Cash Street: high,-74.0,-73.0,-50.0,-50.0,-50.0,1.0,0,4.0,1,0
3,2462807,343731,Cash loans,17887.500,225000.000,225000.00,NaN,225000.000,THURSDAY,12,Y,1.0,NaN,NaN,NaN,XNA,Approved,-77.0,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,18.0,middle,Cash Street: middle,-77.0,-76.0,-59.0,-59.0,-59.0,0.0,0,2.0,1,0
4,2006470,429750,Cash loans,19681.965,270000.000,299938.50,NaN,270000.000,THURSDAY,18,Y,1.0,NaN,NaN,NaN,XNA,Approved,-80.0,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,24.0,middle,Cash Street: middle,-84.0,-83.0,-60.0,-64.0,-64.0,1.0,-1,-1.0,0,1
5,1852667,133853,Consumer loans,4403.565,43606.350,43389.00,4361.850,43606.350,THURSDAY,12,Y,1.0,0.099484,0.142440,0.637949,XAP,Approved,-15.0,Cash through the bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,30.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-14.0,-3.0,-11.0,-11.0,0.0,0,4.0,1,0
6,1663907,107678,Consumer loans,2684.520,29551.050,26451.00,5908.050,29551.050,SUNDAY,10,Y,1.0,0.198844,0.160716,0.715645,XAP,Approved,-13.0,Cash through the bank,XAP,Unaccompanied,Refreshed,Mobile,POS,XNA,Country-wide,92.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-12.0,-1.0,-10.0,-10.0,0.0,0,4.0,1,0
7,1013587,347427,Consumer loans,6155.460,75815.685,60651.00,15164.685,75815.685,TUESDAY,16,Y,1.0,0.217840,0.160716,0.715645,XAP,Approved,-8.0,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,25.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-7.0,3.0,-4.0,-4.0,0.0,0,1.0,1,0
8,1818171,142422,Consumer loans,6800.220,66916.800,67375.80,6696.000,66916.800,THURSDAY,16,Y,1.0,0.098452,0.193330,0.852537,XAP,Approved,-3.0,Cash through the bank,XAP,Unaccompanied,Repeater,Mobile,POS,XNA,Country-wide,93.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-1.0,9.0,0.0,0.0,0.0,0,4.0,1,0
9,2798042,142422,Consumer loans,3982.905,43606.350,39244.50,4361.850,43606.350,THURSDAY,18,Y,1.0,0.108939,0.142440,0.637949,XAP,Approved,-19.0,Cash through the bank,XAP,"Spouse, partner",Repeater,Mobile,POS,XNA,Country-wide,50.0,Connectivity,12.0,middle,POS mobile with interest,NaN,-18.0,-7.0,-18.0,-18.0,0.0,0,4.0,1,0


In [59]:
' previous '
prev['DAYS_LAST_ORIGINAL'] = prev.apply(lambda x: x['DAYS_LAST_DUE'] if x['DAYS_LAST_DUE'] < 99999 else x['DAYS_LAST_DUE_1ST_VERSION'], axis=1)
prev['CNT_ANNUITY'] = prev['AMT_CREDIT'] / prev['AMT_ANNUITY']
prev['AMT_YIELD'] = prev['AMT_CREDIT'] / prev['AMT_APPLICATION']

amt_list = [col for col in prev.columns if ( col.count('CNT_') or col.count('AMT_')) and not(col.count('GOODS'))]
days_list = [col for col in prev.columns if col.count('DAYS_') and (not(col.count('FIRST')) and not(col.count('LAST_DUE')) or not(col.count('TERM')) )]
other_list = [col for col in prev.columns if col.count('SELL')]

amt_sum = prev[[unique_id] + amt_list].groupby(unique_id).sum().reset_index()
days_max = prev[[unique_id] + days_list].groupby(unique_id).max().reset_index()
other_max = prev[[unique_id] + other_list].groupby(unique_id).max().reset_index()

for col in amt_sum[amt_list].columns:
    amt_sum.rename(columns={col:f'{col}_sum'}, inplace=True)
for col in days_max[days_list].columns:
    days_max.rename(columns={col:f'{col}_max'}, inplace=True)
for col in other_max[other_list].columns:
    other_max.rename(columns={col:f'{col}_max'}, inplace=True)
    
print(amt_list)
print(days_list)
print(other_list)

['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'CNT_PAYMENT', 'CNT_ANNUITY', 'AMT_YIELD']
['DAYS_DECISION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'DAYS_LAST_ORIGINAL']
['SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY']


In [58]:
prefix = 'p_'
label = 'DAYS_DECISION'
data = prev[[unique_id, label, 'DAYS_LAST_ORIGINAL'] + amt_list].copy()
data['PREV_CREDIT_TERM'] = data['DAYS_LAST_ORIGINAL'] - data[label]

' 重み付き平均 '
weight_list = [0.99]
feature_list = amt_list + ['PREV_CREDIT_TERM']

df_wmean = prev[unique_id].drop_duplicates().to_frame().copy()

for num in feature_list:
    for weight in weight_list:
        df = data[[unique_id, label, num]].dropna().copy()
        wavg = exp_weight_avg(data=df, level=unique_id, weight=weight, label=label, value=num)
#         result = base.merge(wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
        df_wmean = df_wmean.merge(wavg.to_frame().reset_index(), on=unique_id, how='left')
#         make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)
print(df_wmean.shape)
display(df_wmean.head(20))
del data, df

(338857, 9)


,SK_ID_CURR,AMT_ANNUITY_wg0.99_avg@SK_ID_CURR,AMT_APPLICATION_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_wg0.99_avg@SK_ID_CURR,AMT_DOWN_PAYMENT_wg0.99_avg@SK_ID_CURR,CNT_PAYMENT_wg0.99_avg@SK_ID_CURR,CNT_ANNUITY_wg0.99_avg@SK_ID_CURR,AMT_YIELD_wg0.99_avg@SK_ID_CURR,PREV_CREDIT_TERM_wg0.99_avg@SK_ID_CURR
0,299809,11619.580715,79932.095639,166816.459577,4635.000000,10.655805,11.965356,1.035651,11.215912
1,426902,9287.141720,79243.815606,81999.615300,6996.853436,11.717629,9.099575,1.015953,11.596231
2,157753,10198.649881,170725.790880,171973.925620,2520.000000,24.978202,15.394391,1.006225,17.358293
3,343731,25445.578540,469246.420658,499751.183681,6354.000000,22.903403,17.083551,1.037727,13.095922
4,429750,14584.475342,261778.789988,269260.474134,NaN,14.627486,18.609703,1.037256,19.538260
5,133853,5561.251597,67546.702691,71922.513128,6720.815895,13.625995,11.931070,1.003813,8.064723
6,107678,4213.338620,27327.823515,26912.309409,5316.270175,8.904381,7.485739,0.977288,5.324267
7,347427,8998.167355,162872.486205,153713.182882,15164.685000,16.752132,14.731958,0.879190,11.920219
8,142422,11608.112166,162219.384809,160904.421582,5385.420194,13.568995,11.069367,0.967354,3.784054
9,372255,16547.143749,322677.951805,338967.768117,7916.670274,19.221556,15.821841,0.986336,6.813721


In [60]:
prev_imp_base = amt_sum.merge(days_max, on=unique_id, how='left').merge(df_wmean, on=unique_id, how='left')
result_prev = base.merge(prev_imp_base, on=unique_id, how='left')
result_prev.to_csv('../data/impute_before_prev.csv', index=False)
result_prev.head()

,SK_ID_CURR,TARGET,AMT_ANNUITY_sum,AMT_APPLICATION_sum,AMT_CREDIT_sum,AMT_DOWN_PAYMENT_sum,CNT_PAYMENT_sum,CNT_ANNUITY_sum,AMT_YIELD_sum,DAYS_DECISION_max,DAYS_FIRST_DRAWING_max,DAYS_FIRST_DUE_max,DAYS_LAST_DUE_1ST_VERSION_max,DAYS_LAST_DUE_max,DAYS_TERMINATION_max,DAYS_LAST_ORIGINAL_max,AMT_ANNUITY_wg0.99_avg@SK_ID_CURR,AMT_APPLICATION_wg0.99_avg@SK_ID_CURR,AMT_CREDIT_wg0.99_avg@SK_ID_CURR,AMT_DOWN_PAYMENT_wg0.99_avg@SK_ID_CURR,CNT_PAYMENT_wg0.99_avg@SK_ID_CURR,CNT_ANNUITY_wg0.99_avg@SK_ID_CURR,AMT_YIELD_wg0.99_avg@SK_ID_CURR,PREV_CREDIT_TERM_wg0.99_avg@SK_ID_CURR
0,100002,1,9251.775,179055.00,179055.00,0.00,24.0,19.353584,1.000000,-20.0,NaN,-18.0,4.0,0.0,0.0,0.0,9251.775000,179055.000000,179055.000000,NaN,24.000000,19.353584,1.000000,20.000000
1,100003,0,96631.290,1254908.43,1326899.43,6885.00,30.0,38.108337,3.094615,-24.0,NaN,-23.0,-12.0,-17.0,-17.0,-17.0,36243.684603,476614.186555,504824.188111,6885.000000,9.718255,13.187450,1.038502,7.758597
2,100004,0,5357.250,24282.00,20106.00,4860.00,4.0,3.753045,0.828021,-27.0,NaN,-26.0,-23.0,-24.0,-23.0,-24.0,5357.250000,24282.000000,20106.000000,4860.000000,4.000000,3.753045,0.828021,3.000000
3,100006,0,141907.050,2449829.34,2625259.50,69537.33,132.0,106.603720,6.076104,-6.0,NaN,-5.0,38.0,-5.0,-4.0,1.0,23878.406202,416890.005569,447546.304131,36379.148558,22.350412,18.085388,1.016087,5.930365
4,100007,0,73672.830,903181.50,999832.50,6781.50,121.0,75.864451,6.278136,-12.0,NaN,-11.0,11.0,-11.0,-11.0,11.0,13424.820540,168297.614002,186929.373477,3362.127472,21.534058,13.368336,1.064991,15.761674


In [84]:
' ccb '
# ccb = pd.read_csv('../data/credit_card_balance.csv')
# sys.exit()

amt_list = [col for col in ccb.columns if col.count('AMT_')]
cnt_list = [col for col in ccb.columns if col.count('CNT')]
month_list = [col for col in ccb.columns if col.count('MONTH')]

amt_sum = ccb[[unique_id] + amt_list].groupby(unique_id).sum().reset_index()
cnt_sum = ccb[[unique_id] + cnt_list].groupby(unique_id).sum().reset_index()
month_max = ccb[[unique_id] + month_list].groupby(unique_id).max().reset_index()

for col in amt_sum[amt_list].columns:
    amt_sum.rename(columns={col:f'{col}_sum'}, inplace=True)
for col in cnt_sum[cnt_list].columns:
    cnt_sum.rename(columns={col:f'{col}_sum'}, inplace=True)
for col in month_max[month_list].columns:
    month_max.rename(columns={col:f'{col}_max'}, inplace=True)
    
print(amt_list)
print(cnt_list)
print(month_list)

['AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE']
['CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM']
['MONTHS_BALANCE']


In [85]:
prefix = 'ccb_'
label = 'MONTHS_BALANCE'
data = ccb[[unique_id, label] + amt_list + cnt_list].copy()

' 重み付き平均 '
weight_list = [0.98]
feature_list = amt_list + cnt_list

df_wmean = ccb[unique_id].drop_duplicates().to_frame().copy()

for num in feature_list:
    for weight in weight_list:
        df = data[[unique_id, label, num]].dropna().copy()
        wavg = exp_weight_avg(data=df, level=unique_id, weight=weight, label=label, value=num)
#         result = base.merge(wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
        df_wmean = df_wmean.merge(wavg.to_frame().reset_index(), on=unique_id, how='left')
#         make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)
print(df_wmean.shape)
display(df_wmean.head(20))
del data, df

(103558, 18)


,SK_ID_CURR,AMT_BALANCE_wg0.98_avg@SK_ID_CURR,AMT_CREDIT_LIMIT_ACTUAL_wg0.98_avg@SK_ID_CURR,AMT_DRAWINGS_ATM_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_DRAWINGS_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_DRAWINGS_OTHER_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_DRAWINGS_POS_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_INST_MIN_REGULARITY_wg0.98_avg@SK_ID_CURR,AMT_PAYMENT_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_PAYMENT_TOTAL_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_RECEIVABLE_PRINCIPAL_wg0.98_avg@SK_ID_CURR,AMT_RECIVABLE_wg0.98_avg@SK_ID_CURR,AMT_TOTAL_RECEIVABLE_wg0.98_avg@SK_ID_CURR,CNT_DRAWINGS_ATM_CURRENT_wg0.98_avg@SK_ID_CURR,CNT_DRAWINGS_CURRENT_wg0.98_avg@SK_ID_CURR,CNT_DRAWINGS_OTHER_CURRENT_wg0.98_avg@SK_ID_CURR,CNT_DRAWINGS_POS_CURRENT_wg0.98_avg@SK_ID_CURR,CNT_INSTALMENT_MATURE_CUM_wg0.98_avg@SK_ID_CURR
0,378907,21914.266011,169013.001644,869.398457,2497.822150,0.000000,1628.423694,1831.084544,3894.861123,3894.861123,21197.816072,22037.868156,22037.868156,0.012880,0.221293,0.000000,0.208413,24.168022
1,363914,29320.704912,60241.862150,2834.358252,4102.474541,0.000000,1268.116289,1597.659013,4662.711990,4626.581069,27829.043021,29392.452479,29392.452479,0.282062,0.498276,0.000000,0.216214,52.769915
2,371185,112194.925587,392942.063038,14185.753489,20552.508909,0.000000,6751.101179,5942.159442,22666.064455,22249.974938,107890.687028,111779.714158,111779.714158,0.599085,3.426019,0.000000,2.891002,20.661212
3,337855,194403.698735,204032.349863,3361.801178,16131.608718,0.000000,12769.807541,8862.937493,9814.372112,9116.151441,185821.138555,191454.834832,191454.834832,1.010048,4.705007,0.000000,3.694959,6.434194
4,126868,252308.021905,271662.503102,8947.049861,16931.640326,0.000000,7984.590464,12750.772413,18647.440670,18630.196596,245203.534627,252294.056053,252294.056053,0.390308,1.762708,0.000000,1.372400,68.120924
5,380010,116881.536426,277562.275817,31962.232665,23376.939450,0.000000,515.824110,5278.089032,8461.602933,5467.127493,113629.743624,116045.028187,116045.028187,0.784376,0.638241,0.000000,0.102346,2.975995
6,171320,284970.387324,452186.773006,30774.474489,26892.685114,0.000000,0.000000,13388.998737,19445.739687,15728.839172,278943.834628,284650.957564,284650.957564,0.232752,0.203394,0.000000,0.000000,5.016423
7,118650,18160.944976,97282.243896,1982.272412,2633.424844,651.152433,0.000000,1603.189871,3359.612320,3138.494993,17281.060066,18116.309643,18116.309643,0.044050,0.049616,0.005565,0.000000,48.504948
8,367360,116368.145946,292500.000000,41260.897617,78661.003125,0.000000,37400.105507,6618.275111,90551.969954,90551.969954,112027.432270,113685.360172,113685.360172,1.234300,2.493613,0.000000,1.259313,3.080783
9,203885,76295.024751,128689.612459,10058.195267,16894.230315,0.000000,6836.035048,4088.427366,15940.340254,15940.340254,73460.546865,75643.060943,75643.060943,0.494756,2.063225,0.000000,1.568469,24.043845


In [86]:
ccb_imp_base = amt_sum.merge(cnt_sum, on=unique_id, how='left').merge(df_wmean, on=unique_id, how='left').merge(month_max, on=unique_id, how='left')
result_ccb = base.merge(ccb_imp_base, on=unique_id, how='left')
result_ccb.to_csv('../data/impute_before_ccb.csv', index=False)
result_ccb.head()

,SK_ID_CURR,TARGET,AMT_BALANCE_sum,AMT_CREDIT_LIMIT_ACTUAL_sum,AMT_DRAWINGS_ATM_CURRENT_sum,AMT_DRAWINGS_CURRENT_sum,AMT_DRAWINGS_OTHER_CURRENT_sum,AMT_DRAWINGS_POS_CURRENT_sum,AMT_INST_MIN_REGULARITY_sum,AMT_PAYMENT_CURRENT_sum,AMT_PAYMENT_TOTAL_CURRENT_sum,AMT_RECEIVABLE_PRINCIPAL_sum,AMT_RECIVABLE_sum,AMT_TOTAL_RECEIVABLE_sum,CNT_DRAWINGS_ATM_CURRENT_sum,CNT_DRAWINGS_CURRENT_sum,CNT_DRAWINGS_OTHER_CURRENT_sum,CNT_DRAWINGS_POS_CURRENT_sum,CNT_INSTALMENT_MATURE_CUM_sum,AMT_BALANCE_wg0.98_avg@SK_ID_CURR,AMT_CREDIT_LIMIT_ACTUAL_wg0.98_avg@SK_ID_CURR,AMT_DRAWINGS_ATM_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_DRAWINGS_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_DRAWINGS_OTHER_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_DRAWINGS_POS_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_INST_MIN_REGULARITY_wg0.98_avg@SK_ID_CURR,AMT_PAYMENT_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_PAYMENT_TOTAL_CURRENT_wg0.98_avg@SK_ID_CURR,AMT_RECEIVABLE_PRINCIPAL_wg0.98_avg@SK_ID_CURR,AMT_RECIVABLE_wg0.98_avg@SK_ID_CURR,AMT_TOTAL_RECEIVABLE_wg0.98_avg@SK_ID_CURR,CNT_DRAWINGS_ATM_CURRENT_wg0.98_avg@SK_ID_CURR,CNT_DRAWINGS_CURRENT_wg0.98_avg@SK_ID_CURR,CNT_DRAWINGS_OTHER_CURRENT_wg0.98_avg@SK_ID_CURR,CNT_DRAWINGS_POS_CURRENT_wg0.98_avg@SK_ID_CURR,CNT_INSTALMENT_MATURE_CUM_wg0.98_avg@SK_ID_CURR,MONTHS_BALANCE_max
0,100002,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0.0,1620000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,270000.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,-1.0
4,100007,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
